In [1]:
import warnings
from utils.dataset_MNIST import *
from utils.OpenGAN_arc import *
from utils.visualization import *
from utils.extr_fea import *



plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
config = {
    "dataset": "MNIST",  # 仅使用MNIST数据集
    "closed_classes": [0, 1, 2, 3, 4],  # 闭集类别 (K=5)
    "open_classes": [5, 6, 7, 8, 9],  # 开集类别
    "feature_dim": 512,  # ResNet18特征维度
    "batch_size": 128,
    "lr_classifier": 0.01,  # 分类器学习率
    "lr_gan": 1e-4,  # GAN学习率
    "classifier_epochs": 10,  # 分类器训练轮数
    "gan_epochs": 50,  # GAN训练轮数
    "lambda_g": 0.2,  # 生成样本权重
    "val_ratio": 0.1,  # 验证集比例
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "save_dir": "./mnist_results"  # 结果保存目录
}
# 创建保存目录
os.makedirs(config["save_dir"], exist_ok=True)

In [3]:
if __name__ == "__main__":
    # 加载数据
    train_loader, val_loader, test_loader = load_MNIST(config)

    # 训练闭集分类器
    classifier = train_classifierI(train_loader, val_loader, config)

    # 训练OpenGAN
    discriminator = train_opengan(classifier, train_loader, val_loader, config)

    # 加载最佳判别器
    best_discriminator = Discriminator().to(config["device"])
    best_discriminator.load_state_dict(
        torch.load(os.path.join(config["save_dir"], "best_discriminator.pth")))

    # 可视化结果
    visualize_results(classifier, best_discriminator, test_loader["closed"], test_loader["open"], config)

    # 最终测试评估
    test_auroc = evaluate_opengan(best_discriminator, classifier, test_loader["closed"], test_loader["open"], config)
    print(f"\n最终测试集AUROC: {test_auroc:.4f}")

加载MNIST数据集...
训练集: 27537闭集样本
验证集: 3059闭集 + 3059开集样本
测试集: 5139闭集 + 4861开集样本

训练闭集分类器...


E:\Anaconda\envs\dl_torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Anaconda\envs\dl_torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
分类器 Epoch 1: 100%|██████████| 216/216 [00:43<00:00,  4.94it/s]


Epoch 1: 训练损失=0.2222, 验证准确率=0.9582


分类器 Epoch 2: 100%|██████████| 216/216 [00:38<00:00,  5.63it/s]


Epoch 2: 训练损失=0.0383, 验证准确率=0.9928


...

Epoch 49: D_loss=-3.2236, G_loss=16.1181, Val AUROC=0.6353


OpenGAN Epoch 50: 100%|██████████| 216/216 [00:19<00:00, 10.94it/s]


Epoch 50: D_loss=-3.2236, G_loss=16.1181, Val AUROC=0.6378
最佳验证AUROC: 0.9290

可视化结果...
测试集AUROC: 0.9375
可视化特征空间...
可视化决策边界...


E:\Projects\Learn\OpenGAN\utils\visualization.py:196: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(features_2d[labels == 1, 0], features_2d[labels == 1, 1],



最终测试集AUROC: 0.9375
